In [45]:
import sqlite3
import pandas as pd

# Kết nối database (hoặc tạo mới)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng course
cursor.execute('''CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
)''')
cursor.executemany('INSERT INTO course (id, course_name) VALUES (?, ?)', [
    (12, 'Giai tich'),
    (34, 'Thong ke'),
    (26, 'Tin hoc')
])

# Tạo bảng student
cursor.execute('''CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)''')
cursor.executemany('INSERT INTO student (student_id, name, class, course_id, score) VALUES (?, ?, ?, ?, ?)', [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
])

In [54]:
# Kết nối hai bảng bằng tích Descartes (CROSS JOIN)
cross_join_query = "SELECT * FROM student CROSS JOIN course"
cross_join_result = pd.read_sql(cross_join_query, conn)

print("Kết nối hai bảng bằng tích Descartes:")
print(cross_join_result)


Kết nối hai bảng bằng tích Descartes:
    student_id               name     class  course_id  score  \
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7   
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7   
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7   
3            2       Tran Thi Lan   Kinh Te       34.0    9.2   
4            2       Tran Thi Lan   Kinh Te       34.0    9.2   
5            2       Tran Thi Lan   Kinh Te       34.0    9.2   
6            3       Pham Van Nam  Toan Tin        NaN    7.9   
7            3       Pham Van Nam  Toan Tin        NaN    7.9   
8            3       Pham Van Nam  Toan Tin        NaN    7.9   
9            4     Le Thanh Huyen  Toan Tin        NaN    7.2   
10           4     Le Thanh Huyen  Toan Tin        NaN    7.2   
11           4     Le Thanh Huyen  Toan Tin        NaN    7.2   
12           5        Vu Quoc Anh  May Tinh        NaN    8.0   
13           5        Vu Quoc Anh  May Tinh        N

In [55]:
# Cập nhật course_id bị thiếu
cursor.execute('''UPDATE student SET course_id = NULL WHERE course_id NOT IN (SELECT id FROM course)''')

# Kết nối bảng student và course với các loại JOIN
joins = {
    "INNER JOIN": "SELECT * FROM student INNER JOIN course ON student.course_id = course.id",
    "LEFT JOIN": "SELECT * FROM student LEFT JOIN course ON student.course_id = course.id",
    "RIGHT JOIN (Simulation)": "SELECT * FROM course LEFT JOIN student ON student.course_id = course.id",
    "FULL OUTER JOIN (Simulation)": "SELECT * FROM student LEFT JOIN course ON student.course_id = course.id UNION SELECT * FROM course LEFT JOIN student ON student.course_id = course.id"
}
for join_type, query in joins.items():
    print(f"\n{join_type} Result:")
    print(pd.read_sql(query, conn))


INNER JOIN Result:
   student_id               name     class  course_id  score  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   
1           2       Tran Thi Lan   Kinh Te         34    9.2   
2           7      Bui Tien Dung   Kinh Te         34    9.2   

       graduation_date  id course_name  
0  2025-04-13 07:44:16  12   Giai tich  
1  2025-05-13 07:44:16  34    Thong ke  
2  2025-10-10 07:44:16  34    Thong ke  

LEFT JOIN Result:
   student_id               name     class  course_id  score  \
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7   
1           2       Tran Thi Lan   Kinh Te       34.0    9.2   
2           3       Pham Van Nam  Toan Tin        NaN    7.9   
3           4     Le Thanh Huyen  Toan Tin        NaN    7.2   
4           5        Vu Quoc Anh  May Tinh        NaN    8.0   
5           6     Dang Thuy Linh  May Tinh        NaN    5.5   
6           7      Bui Tien Dung   Kinh Te       34.0    9.2   
7           8        Ho Ngoc

In [47]:

# Thống kê theo lớp học
class_stats = df.groupby('class').agg({'student_id': 'count', 'score': 'mean'}).rename(columns={'student_id': 'total_students', 'score': 'avg_score'})
print("Thống kê theo lớp:")
print(class_stats)

Thống kê theo lớp:
          total_students  avg_score
class                              
Kinh Te                3   8.533333
May Tinh               4   6.800000
Toan Tin               3   7.966667


In [48]:
# Thống kê theo môn học
course_stats = df.groupby('course_name').agg({'student_id': 'count', 'score': 'mean'}).rename(columns={'student_id': 'total_students', 'score': 'avg_score'})
print("Thống kê theo môn học:")
print(course_stats)

Thống kê theo môn học:
             total_students  avg_score
course_name                           
Giai tich                 1        6.7
Thong ke                  2        9.2


In [49]:
# Phân loại điểm
classification_query = '''SELECT course.course_name, AVG(student.score) AS avg_score, 
CASE 
    WHEN AVG(student.score) >= 9.0 THEN 'Xuất sắc'
    WHEN AVG(student.score) >= 6.0 THEN 'Tốt'
    ELSE 'Kém' 
END AS classification
FROM student LEFT JOIN course ON student.course_id = course.id GROUP BY course.course_name'''
print("Phân loại điểm:")
print(pd.read_sql(classification_query, conn))

Phân loại điểm:
  course_name  avg_score classification
0        None   7.371429            Tốt
1   Giai tich   6.700000            Tốt
2    Thong ke   9.200000       Xuất sắc


In [50]:
# Xếp hạng sinh viên theo điểm số
df_ranked = df.sort_values(by='score', ascending=False)
print("Xếp hạng sinh viên theo điểm số:")
print(df_ranked.head(3))  # Top 3
print(df_ranked.tail(3))  # Bottom 3

Xếp hạng sinh viên theo điểm số:
   student_id           name     class course_name  score
1           2   Tran Thi Lan   Kinh Te    Thong ke    9.2
6           7  Bui Tien Dung   Kinh Te    Thong ke    9.2
7           8    Ho Ngoc Mai  Toan Tin        None    8.8
   student_id               name     class course_name  score
9          10       Cao Thi Hanh  May Tinh        None    7.0
0           1  Nguyen Minh Hoang  May Tinh   Giai tich    6.7
5           6     Dang Thuy Linh  May Tinh        None    5.5


In [51]:
# Xếp hạng sinh viên theo lớp học
class_rank_query = '''SELECT class, name, score, RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank FROM student'''
print("Xếp hạng sinh viên theo lớp học:")
print(pd.read_sql(class_rank_query, conn))

Xếp hạng sinh viên theo lớp học:
      class               name  score  rank
0   Kinh Te       Tran Thi Lan    9.2     1
1   Kinh Te      Bui Tien Dung    9.2     1
2   Kinh Te     Duong Huu Phuc    7.2     3
3  May Tinh        Vu Quoc Anh    8.0     1
4  May Tinh       Cao Thi Hanh    7.0     2
5  May Tinh  Nguyen Minh Hoang    6.7     3
6  May Tinh     Dang Thuy Linh    5.5     4
7  Toan Tin        Ho Ngoc Mai    8.8     1
8  Toan Tin       Pham Van Nam    7.9     2
9  Toan Tin     Le Thanh Huyen    7.2     3


In [52]:
# Xếp hạng sinh viên theo mã môn học
course_rank_query = '''SELECT course_id, name, score, RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank FROM student'''
print("Xếp hạng sinh viên theo mã môn học:")
print(pd.read_sql(course_rank_query, conn))


Xếp hạng sinh viên theo mã môn học:
   course_id               name  score  rank
0        NaN        Ho Ngoc Mai    8.8     1
1        NaN        Vu Quoc Anh    8.0     2
2        NaN       Pham Van Nam    7.9     3
3        NaN     Le Thanh Huyen    7.2     4
4        NaN     Duong Huu Phuc    7.2     4
5        NaN       Cao Thi Hanh    7.0     6
6        NaN     Dang Thuy Linh    5.5     7
7       12.0  Nguyen Minh Hoang    6.7     1
8       34.0       Tran Thi Lan    9.2     1
9       34.0      Bui Tien Dung    9.2     1


In [53]:
# Bổ sung trường graduation_date
cursor.execute("ALTER TABLE student ADD COLUMN graduation_date DATETIME")

# Tính ngày tốt nghiệp giả định
import datetime
for idx, row in df_ranked.iterrows():
    rank = df_ranked[df_ranked['student_id'] == row['student_id']].index[0] + 1
    grad_date = datetime.datetime.now() + datetime.timedelta(days=int(rank) * 30)
    cursor.execute("UPDATE student SET graduation_date = ? WHERE student_id = ?", (grad_date.strftime("%Y-%m-%d %H:%M:%S"), row['student_id']))


# Hiển thị dữ liệu sau khi cập nhật
final_query = "SELECT * FROM student"
final_df = pd.read_sql(final_query, conn)
print(final_df)


   student_id               name     class  course_id  score  \
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7   
1           2       Tran Thi Lan   Kinh Te       34.0    9.2   
2           3       Pham Van Nam  Toan Tin        NaN    7.9   
3           4     Le Thanh Huyen  Toan Tin        NaN    7.2   
4           5        Vu Quoc Anh  May Tinh        NaN    8.0   
5           6     Dang Thuy Linh  May Tinh        NaN    5.5   
6           7      Bui Tien Dung   Kinh Te       34.0    9.2   
7           8        Ho Ngoc Mai  Toan Tin        NaN    8.8   
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2   
9          10       Cao Thi Hanh  May Tinh        NaN    7.0   

       graduation_date  
0  2025-04-13 07:44:16  
1  2025-05-13 07:44:16  
2  2025-06-12 07:44:16  
3  2025-07-12 07:44:16  
4  2025-08-11 07:44:16  
5  2025-09-10 07:44:16  
6  2025-10-10 07:44:16  
7  2025-11-09 07:44:16  
8  2025-12-09 07:44:16  
9  2026-01-08 07:44:16  
